In [1]:
import nd2
import xml.etree.ElementTree as ET

In [2]:
# Input files
nd2_microscope_images = '20250220_Galvano-Full-Plate.nd2'
positions_file = 'multipoints.xml'

In [3]:
# Read nd2 file 
with nd2.ND2File(nd2_microscope_images) as f:
    ome = f.ome_metadata()
    xml = ome.to_xml()

# Write the nd2 XML string to a file
with open("microscope_image_data.xml", "w", encoding="utf-8") as xml_file:
    xml_file.write(xml)

ND2: Finding the image name, x-position, and y-position in the microscope images file

In [4]:
# Parse the nd2 XML file
tree_images = ET.parse('microscope_image_data.xml')
root_images = tree_images.getroot()

# Define the namespace dictionary
ns = {'ome': 'http://www.openmicroscopy.org/Schemas/OME/2016-06'}

# List to hold results
planes = []

# Iterate over each Image element
for image in root_images.findall('ome:Image', ns):
    image_name = image.get('Name')
    # Find the first Plane element within the Pixels element
    plane = image.find('.//ome:Plane', ns)
    if plane is not None:
        posX = float(plane.get('PositionX'))
        posY = float(plane.get('PositionY'))
        planes.append({
            'image_name': image_name,
            'posX': posX,
            'posY': posY
        })

In [5]:
# Print the results
for item in planes:
    print(item)


{'image_name': '20250220_Galvano-Full-Plate (Series 0)', 'posX': 42151.1, 'posY': -22766.5}
{'image_name': '20250220_Galvano-Full-Plate (Series 1)', 'posX': 41225.0, 'posY': -22449.5}
{'image_name': '20250220_Galvano-Full-Plate (Series 2)', 'posX': 41365.7, 'posY': -21952.4}
{'image_name': '20250220_Galvano-Full-Plate (Series 3)', 'posX': 40293.8, 'posY': -13686.6}
{'image_name': '20250220_Galvano-Full-Plate (Series 4)', 'posX': 40339.1, 'posY': -11346.8}
{'image_name': '20250220_Galvano-Full-Plate (Series 5)', 'posX': 39000.4, 'posY': -11719.0}
{'image_name': '20250220_Galvano-Full-Plate (Series 6)', 'posX': 41159.7, 'posY': -4477.3}
{'image_name': '20250220_Galvano-Full-Plate (Series 7)', 'posX': 41577.1, 'posY': -2551.5}
{'image_name': '20250220_Galvano-Full-Plate (Series 8)', 'posX': 38947.2, 'posY': -2794.1}
{'image_name': '20250220_Galvano-Full-Plate (Series 9)', 'posX': 40389.6, 'posY': 4829.5}
{'image_name': '20250220_Galvano-Full-Plate (Series 10)', 'posX': 41157.4, 'posY': 46

Multipoint: Finding the positions name, x-position, and y-position in the multipoints file

In [6]:
# Parse multipoints XML file 
tree_positions = ET.parse(positions_file)
root_positions = tree_positions .getroot()

# Find the container of the point elements (named 'no_name')
no_name = root_positions.find('no_name')

# List to hold the results
points = []

# Iterate over all child elements (e.g., Point00000, Point00001, etc.)
for point in no_name:
    # Extract the 'value' attribute from the sub-elements: strName, dXPosition, dYPosition
    name_elem = point.find('strName')
    x_elem = point.find('dXPosition')
    y_elem = point.find('dYPosition')
    
    if name_elem is not None and x_elem is not None and y_elem is not None:
        strName = name_elem.get('value')
        dX = float(x_elem.get('value'))
        dY = float(y_elem.get('value'))
        points.append({
            'strName': strName,
            'dX': dX,
            'dY': dY
        })

In [7]:
# Print the results
for p in points:
    print(p)

{'strName': 'B2_1', 'dX': 42151.100000000006, 'dY': -22766.5}
{'strName': 'B2_2', 'dX': 41225.0, 'dY': -22449.5}
{'strName': 'B2_3', 'dX': 41365.700000000004, 'dY': -21952.4}
{'strName': 'C2_1', 'dX': 40293.8, 'dY': -13686.6}
{'strName': 'C2_2', 'dX': 40339.100000000006, 'dY': -11346.800000000001}
{'strName': 'C2_3', 'dX': 39000.4, 'dY': -11719.0}
{'strName': 'D2_1', 'dX': 41159.700000000004, 'dY': -4477.3}
{'strName': 'D2_2', 'dX': 41577.100000000006, 'dY': -2551.5}
{'strName': 'D2_3', 'dX': 38947.200000000004, 'dY': -2794.1000000000004}
{'strName': 'E2_1', 'dX': 40389.600000000006, 'dY': 4829.5}
{'strName': 'E2_2', 'dX': 41157.4, 'dY': 4672.5}
{'strName': 'E2_3', 'dX': 42354.0, 'dY': 5574.1}
{'strName': 'F2_1', 'dX': 39484.200000000004, 'dY': 14312.0}
{'strName': 'F2_2', 'dX': 39096.700000000004, 'dY': 15147.0}
{'strName': 'F2_3', 'dX': 40666.700000000004, 'dY': 15376.300000000001}
{'strName': 'B3_1', 'dX': 32581.800000000003, 'dY': -22678.7}
{'strName': 'B3_2', 'dX': 33403.200000000

Find matching positions between the image file and multipoints file

In [8]:
error_margin = 1e-3
matches = []

# Compare each plane from the first XML with each point from the second XML.
for plane in planes:
    for point in points:
        # Check if both X and Y differences are within the margin.
        if abs(plane['posX'] - point['dX']) <= error_margin and abs(plane['posY'] - point['dY']) <= error_margin:
            matches.append({
                'image_name': plane['image_name'],
                'point_name': point['strName'],
                'posX': plane['posX'],
                'posY': plane['posY'],
                'dX': point['dX'],
                'dY': point['dY']
            })

In [9]:
# Print the matching results
print("Matching positions (within margin of error):")
for match in matches:
    print(match)

Matching positions (within margin of error):
{'image_name': '20250220_Galvano-Full-Plate (Series 0)', 'point_name': 'B2_1', 'posX': 42151.1, 'posY': -22766.5, 'dX': 42151.100000000006, 'dY': -22766.5}
{'image_name': '20250220_Galvano-Full-Plate (Series 1)', 'point_name': 'B2_2', 'posX': 41225.0, 'posY': -22449.5, 'dX': 41225.0, 'dY': -22449.5}
{'image_name': '20250220_Galvano-Full-Plate (Series 2)', 'point_name': 'B2_3', 'posX': 41365.7, 'posY': -21952.4, 'dX': 41365.700000000004, 'dY': -21952.4}
{'image_name': '20250220_Galvano-Full-Plate (Series 3)', 'point_name': 'C2_1', 'posX': 40293.8, 'posY': -13686.6, 'dX': 40293.8, 'dY': -13686.6}
{'image_name': '20250220_Galvano-Full-Plate (Series 4)', 'point_name': 'C2_2', 'posX': 40339.1, 'posY': -11346.8, 'dX': 40339.100000000006, 'dY': -11346.800000000001}
{'image_name': '20250220_Galvano-Full-Plate (Series 5)', 'point_name': 'C2_3', 'posX': 39000.4, 'posY': -11719.0, 'dX': 39000.4, 'dY': -11719.0}
{'image_name': '20250220_Galvano-Full-Pla